* Ray RLlib 노트북

필요 패키지 삽입

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd

from horcrux_terrain_v1.envs import SandWorld
from horcrux_terrain_v1.envs import PlaneJoyWorld

import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig

from ray.tune.registry import register_env


Gym -> Rllib Env 등록

In [ ]:
register_env("plane-v2", lambda config: PlaneJoyWorld())

학습 알고리즘 설정

In [ ]:
config = PPOConfig()
# Activate new API stack.
config.api_stack(
    enable_rl_module_and_learner=True,
    enable_env_runner_and_connector_v2=True,
)
config.environment("sand-v1")
config.framework("torch")
config.resources(num_gpus=1, num_cpus_for_main_process=16)
config.env_runners(num_env_runners=16)
config.training(
    gamma=0.99, 
    lr=0.0001, 
    # kl_coeff=0.3, 
    train_batch_size_per_learner=32768,              
    sgd_minibatch_size=16384, 
)

# See model catalog for more options.
# https://docs.ray.io/en/latest/rllib/rllib-models.html
config.model["fcnet_hiddens"] = [512, 512, 512, 512, 512]
config.model["use_lstm"] = True
config.model["lstm_cell_size"] = 2048

algo = config.build()


학습 시작

In [ ]:
n_iter = 2500

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}th iteration done")

algo.save("512x5_2048_lstm-3")

다시 로드하기

In [ ]:
algo.restore("512x5_2048_lstm-2")